In [1]:
import cv2
import numpy as np

In [2]:
cd D:\ML\EVA\JEDI\S13

D:\ML\EVA\JEDI\S13


In [3]:
ls

 Volume in drive D has no label.
 Volume Serial Number is 1C8B-70E8

 Directory of D:\ML\EVA\JEDI\S13

24-10-2020  05:09 PM    <DIR>          .
24-10-2020  05:09 PM    <DIR>          ..
13-06-2019  07:33 PM               705 coco.names
04-06-2019  03:38 PM         2,904,826 room_ser.jpg
16-07-2020  09:47 PM             1,889 yolo_object_detection.py
24-10-2020  05:03 PM         2,890,443 yolo_object_detection.zip
21-06-2019  12:43 PM             8,342 yolov3.cfg
24-10-2020  05:08 PM       248,007,048 yolov3.weights
               6 File(s)    253,813,253 bytes
               2 Dir(s)  890,069,729,280 bytes free


In [4]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [5]:
img = cv2.imread("WIN_20201024_17_13_06_Pro.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

In [6]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [7]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [8]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [ ]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()